In [1]:
# Brian-setup
from brian2 import NeuronGroup, BrianLogger, start_scope, run, StateMonitor, SpikeMonitor, stop, rand
from brian2.units import ms, volt, mV, amp, mA, siemens, Hz, ohm, uF
from brian2 import figure, grid, axhline, xlabel, ylabel, plot, legend, title, subplot, zeros, ones, arange, xticks, xlim, ylim

from brian2 import prefs, BrianLogger
prefs.logging.file_log_level = 'WARNING'
BrianLogger.file_handler.setLevel(prefs.logging.file_log_level)

# Scientific libs:
import numpy as np
%matplotlib inline

In [2]:
def perform_single_neuron_test_with_current_for_1_ms(a, b, I_ext=0.1*mA, do_plot=True, print_info=True):
    start_scope()

    tau = 1*ms

    eqs = '''
    dv/dt = ((0.04*v**2)/mV + 5*v + mV*140 - u)/tau + I/uF : volt
    du/dt = a * (b*v - u)/tau : volt
    I : amp
    '''

    c = -65*mV; d = 8*mV;

    G = NeuronGroup(1, eqs, threshold='v>=30*mV', reset='''v=c; u=u+d''', method='euler')
    G.v[0] = c
    G.I[0] = 0

    statemon_v = StateMonitor(G[:], 'v', record=True)
    statemon_u = StateMonitor(G[:], 'u', record=True)
    spikemon = SpikeMonitor(G[:], variables='v')
    
    # stabilize
    t1 = 20*ms
    run(t1)
    # run for 1 ms with large current, triggering a burst
    G.I[0] = I_ext
    run(1*ms)
    G.I[0] = 0

    t2 = 50*tau
    run(t2)
    run_time = t1 + t2 + 1*ms
    firing_rate = spikemon.num_spikes/(run_time)

    # create spike plots
    if(do_plot):
        figure()
        grid(True)
        axhline(30, ls='-', c='lightgray', lw=3)
        plot(statemon_v.t/tau, statemon_v.v.T/tau, '-', label='v')
        plot(statemon_u.t/tau, statemon_u.u.T/tau, '--', label='u')
        xlabel('Time (ms)')
        ylabel('v (mV)')
        legend(['spike threshold', 'v', 'u'])
        title('a='+"{:5.4f}".format(a)+
              ', b='+"{:4.3f}".format(b)+
              ', I_ext='+str(I_ext));
    if(print_info):
        print('a='+"{:6.5f}".format(a)+
              ', b='+"{:4.3f}".format(b)+
              ', I_ext='+str(I_ext));
        
    stop()
    
    return firing_rate;

In [3]:
def perform_single_neuron_test(a, b, I_ext=0.*mA, do_plot=True, print_info=True, timesteps=4000):
    start_scope()

    tau = 1*ms

    eqs = '''
    dv/dt = ((0.04*v**2)/mV + 5*v + mV*140 - u)/tau + I/uF : volt
    du/dt = a * (b*v - u)/tau : volt
    I : amp
    '''

    c = -65*mV; d = 8*mV;

    G = NeuronGroup(1, eqs, threshold='v>=30*mV', reset='''v=c; u=u+d''', method='euler')
    run(20*ms)
    
    statemon_v = StateMonitor(G[:], 'v', record=True)
    statemon_u = StateMonitor(G[:], 'u', record=True)
    spikemon = SpikeMonitor(G[:], variables='v')

    G.I[0] = I_ext
    run_time = timesteps*tau
    run(run_time)
    firing_rate = spikemon.num_spikes/(run_time)

    # create spike plots
    if(do_plot):
        figure()
        grid(True)
        axhline(30, ls='-', c='lightgray', lw=3)
        plot(statemon_v.t/tau, statemon_v.v.T/tau, '-', label='v')
        plot(statemon_u.t/tau, statemon_u.u.T/tau, '--', label='u')
        xlabel('Time (ms)')
        ylabel('v (mV)')
        legend(['spike threshold', 'v', 'u'])
        title('a='+"{:5.4f}".format(a)+
              ', b='+"{:4.3f}".format(b)+
              ', I_ext='+str(I_ext)+
              ', firing rate: '+str(firing_rate));
    if(print_info):
        print('a='+"{:6.5f}".format(a)+
              ', b='+"{:4.3f}".format(b)+
              ', I_ext='+str(I_ext)+
              ', firing rate: '+str(firing_rate));
    return firing_rate;

    stop()

In [4]:
import re

def parse_values_from_logfile(fname):
    with open(fname) as file:
        file_contents = file.read()

    tmp = file_contents.split('\n')
    
    a=[]; b=[]; I_ext=[]; rate=[];
    
    parsed_str_ctr = 0
    for res_str in tmp:
        if(len(res_str) > 0):
            tmp_vars = re.split(', firing rate: |,', res_str)
            a += [float(tmp_vars[0].split('=')[1])]
            b += [float(tmp_vars[1].split('=')[1])]
            I_ext += [tmp_vars[2].split('=')[1]]
            rate += [float(tmp_vars[3].split(' ')[0])]

            parsed_str_ctr += 1
    
    return [a, b, I_ext, rate]

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def neuron_plot_3d(neuron_data):
    [a, b, I_ext, rate] = neuron_data
    
    fig = figure(figsize=(9, 4))
    ax = fig.gca(projection='3d')
    ax.set_zlabel('Avg. firing rate ($Hz$)')
    ax.set_xlabel('$a$-value')
    ax.set_ylabel('$b$-value')
    grid(True)
    
    ax.scatter(a, b, rate)
    
    legend(['Average neuron firing rate'])
    title('Effect of $a$ and $b$ on average network firing rate');